In [1]:
from topicsfinder import TopicsFinder

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kapangyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.
